In [1]:
from PIL import Image, ImageEnhance, ImageFilter  
import os
from glob import glob

import pandas as pd
import numpy as np
import random

In [2]:
def create_folder(path, folder = ""):
    
    final_path = os.path.join(path, folder)
    
    if not os.path.isdir(final_path):
        os.makedirs (final_path)
    
    return final_path

In [3]:
def images_count(path_folder):
    lst = os.listdir(path_folder) 
    count = len(lst)
    return count

In [4]:
# https://www.codementor.io/@isaib.cicourel/image-manipulation-in-python-du1089j1u

# Return color value depending on quadrant and saturation
def get_saturation(value, quadrant):
  if value > 223:
    return 255
  elif value > 159:
    if quadrant != 1:
      return 255

    return 0
  elif value > 95:
    if quadrant == 0 or quadrant == 3:
      return 255

    return 0
  elif value > 32:
    if quadrant == 1:
      return 255

    return 0
  else:
    return 0

# Get the pixel from the given image
def get_pixel(image, i, j):
  # Inside image bounds?
  width, height = image.size
  if i > width or j > height:
    return None

  # Get Pixel
  pixel = image.getpixel((i, j))
  return pixel

# Create a dithered version of the image
def convert_dithering(image):
    # Get size
    width, height = image.size

    # Create new Image and a Pixel Map
    new = create_image_white(width, height)
    pixels = new.load()

    # Transform to half tones
    for i in range(0, width, 2):
        for j in range(0, height, 2):
          # Get Pixels
          p1 = get_pixel(image, i, j)
          p2 = get_pixel(image, i, j + 1)
          p3 = get_pixel(image, i + 1, j)
          p4 = get_pixel(image, i + 1, j + 1)

          # Color Saturation by RGB channel
          red   = (p1[0] + p2[0] + p3[0] + p4[0]) / 4
          green = (p1[1] + p2[1] + p3[1] + p4[1]) / 4
          blue  = (p1[2] + p2[2] + p3[2] + p4[2]) / 4

          # Results by channel
          r = [0, 0, 0, 0]
          g = [0, 0, 0, 0]
          b = [0, 0, 0, 0]

          # Get Quadrant Color
          for x in range(0, 4):
            r[x] = get_saturation(red, x)
            g[x] = get_saturation(green, x)
            b[x] = get_saturation(blue, x)

          # Set Dithered Colors
          pixels[i, j]         = (r[0], g[0], b[0])
          pixels[i, j + 1]     = (r[1], g[1], b[1])
          pixels[i + 1, j]     = (r[2], g[2], b[2])
          pixels[i + 1, j + 1] = (r[3], g[3], b[3])

    # Return new image
    return new

In [5]:
def convert_brightness(image, brightness):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness)
    

In [6]:
def image_name_brightness(name, brightness):
    return ''.join([name, "_", f"{brightness}".replace('.', '_')])

In [7]:
def image_name_dithering(name):
    return ''.join([name, "_dither"])

In [8]:
def image_name_low_quality(name):
    return ''.join([name, "_low"])

In [9]:
def save_image(img, path, name, fillcolor,  quality=75, extension='png'):

    
    img = img.rotate(random.randint(5, 175), Image.NEAREST, expand = False, fillcolor = fillcolor)
    img.save(f"{path}/{name}.{extension}", quality=quality)

In [10]:
# Create a new image with the given size
def create_image_white(i, j):
  image = Image.new("RGB", (i, j), "white")
  return image

In [11]:
def create_image(path, name, r, g, b, override = False, width = 128, height = 128, brightness = 1, extension='png'):
    print(path, name, r, g, b, brightness)
    create_folder(path)
    
    name_brightness = image_name_brightness(name, brightness)
    name_dithering = image_name_dithering(name_brightness)
    name_low_quality = image_name_low_quality(name_brightness)
    
    if glob(f"{path}/{image_name_brightness}.{extension}"):
        #print("already exists")
        if override:
            pass
            
    else:
        #print("let's go !")
        img  = Image.new( mode = "RGB", size = (width, height), color = (r, g, b) )
        
        img = convert_brightness(img, brightness)
        save_image(img, path, name_brightness, (r, g, b))
        
         
        save_image(img, path, name_low_quality, (r, g, b), 1)
        
        # Baisse les perfs
        #img = convert_dithering(img)
        #save_image(img, path, name_dithering, (r, g, b))
        
        
    

In [12]:

df = pd.read_csv("./dataset/html_color_names.csv", sep=";")

df.head()

,color,enable,color_name,hex_code,r,g,b,remark
0,red,1,IndianRed,#CD5C5C,205,92,92,NaN
1,red,1,LightCoral,#F08080,240,128,128,NaN
2,red,1,Salmon,#FA8072,250,128,114,NaN
3,red,0,DarkSalmon,#E9967A,233,150,122,NaN
4,red,0,LightSalmon,#FFA07A,255,160,122,NaN


In [13]:
images_folder = "./dataset/images/"

In [14]:

for index, row in df.iterrows():
    for i in range(9, 14):
        if row['enable'] == 1:
            create_image(
                f"{images_folder}{row['color']}", 
                row['color_name'], 
                r = row['r'], 
                g = row['g'], 
                b = row['b'], 
                brightness=i/10)
    

./dataset/images/red IndianRed 205 92 92 0.9
./dataset/images/red IndianRed 205 92 92 1.0
./dataset/images/red IndianRed 205 92 92 1.1
./dataset/images/red IndianRed 205 92 92 1.2
./dataset/images/red IndianRed 205 92 92 1.3
./dataset/images/red LightCoral 240 128 128 0.9
./dataset/images/red LightCoral 240 128 128 1.0
./dataset/images/red LightCoral 240 128 128 1.1
./dataset/images/red LightCoral 240 128 128 1.2
./dataset/images/red LightCoral 240 128 128 1.3
./dataset/images/red Salmon 250 128 114 0.9
./dataset/images/red Salmon 250 128 114 1.0
./dataset/images/red Salmon 250 128 114 1.1
./dataset/images/red Salmon 250 128 114 1.2
./dataset/images/red Salmon 250 128 114 1.3
./dataset/images/red Crimson 220 20 60 0.9
./dataset/images/red Crimson 220 20 60 1.0
./dataset/images/red Crimson 220 20 60 1.1
./dataset/images/red Crimson 220 20 60 1.2
./dataset/images/red Crimson 220 20 60 1.3
./dataset/images/red Red 255 0 0 0.9
./dataset/images/red Red 255 0 0 1.0
./dataset/images/red Red 2

./dataset/images/purple MediumSlateBlue 123 104 238 1.2
./dataset/images/purple MediumSlateBlue 123 104 238 1.3
./dataset/images/green GreenYellow 173 255 47 0.9
./dataset/images/green GreenYellow 173 255 47 1.0
./dataset/images/green GreenYellow 173 255 47 1.1
./dataset/images/green GreenYellow 173 255 47 1.2
./dataset/images/green GreenYellow 173 255 47 1.3
./dataset/images/green Chartreuse 127 255 0 0.9
./dataset/images/green Chartreuse 127 255 0 1.0
./dataset/images/green Chartreuse 127 255 0 1.1
./dataset/images/green Chartreuse 127 255 0 1.2
./dataset/images/green Chartreuse 127 255 0 1.3
./dataset/images/green LawnGreen 124 252 0 0.9
./dataset/images/green LawnGreen 124 252 0 1.0
./dataset/images/green LawnGreen 124 252 0 1.1
./dataset/images/green LawnGreen 124 252 0 1.2
./dataset/images/green LawnGreen 124 252 0 1.3
./dataset/images/green Lime 0 255 0 0.9
./dataset/images/green Lime 0 255 0 1.0
./dataset/images/green Lime 0 255 0 1.1
./dataset/images/green Lime 0 255 0 1.2
./d

./dataset/images/brown Maroon 128 0 0 1.1
./dataset/images/brown Maroon 128 0 0 1.2
./dataset/images/brown Maroon 128 0 0 1.3
./dataset/images/white White 255 255 255 0.9
./dataset/images/white White 255 255 255 1.0
./dataset/images/white White 255 255 255 1.1
./dataset/images/white White 255 255 255 1.2
./dataset/images/white White 255 255 255 1.3
./dataset/images/white Snow 255 250 250 0.9
./dataset/images/white Snow 255 250 250 1.0
./dataset/images/white Snow 255 250 250 1.1
./dataset/images/white Snow 255 250 250 1.2
./dataset/images/white Snow 255 250 250 1.3
./dataset/images/white GhostWhite 248 248 255 0.9
./dataset/images/white GhostWhite 248 248 255 1.0
./dataset/images/white GhostWhite 248 248 255 1.1
./dataset/images/white GhostWhite 248 248 255 1.2
./dataset/images/white GhostWhite 248 248 255 1.3
./dataset/images/white WhiteSmoke 245 245 245 0.9
./dataset/images/white WhiteSmoke 245 245 245 1.0
./dataset/images/white WhiteSmoke 245 245 245 1.1
./dataset/images/white WhiteS

In [15]:
#files_count(images_folder+'yellow')

max_images_count = 2000

for c in next(os.walk(images_folder))[1]:
    full_path = images_folder+c #images_folder+c
    img_count = images_count(full_path) # files_color_count = 
    print(c, img_count)
    
    for i in range(img_count, max_images_count):
        image_to_copy = os.listdir(full_path)[random.randint(0, img_count-1)]
        image_name_to_copy, image_extension = os.path.splitext(image_to_copy)
        
        img_original = Image.open(full_path+'/'+image_to_copy)
        width, height = img_original.size
        
        colors = img_original.getpixel((1, 1)) #((random.randint(0, width-1), random.randint(0, height-1)))
        r = colors[0]
        g = colors[1]
        b = colors[2]
        

        create_image(
            full_path, 
            image_name_to_copy+'_'+str(i), 
            r = r, 
            g = g, 
            b = b, 
            brightness=random.randint(9, 14)/10)        
        
        
        print(c, image_to_copy, image_name_to_copy, image_extension, r, g, b)
        
        if images_count(full_path) >= max_images_count:
            break

'''
    print(c, files_count(images_folder+'yellow')) # 
    

    files_count = files_count(images_folder+c)
    
    for i in range(files_count(images_folder + c), 1200):
        print(c, os.listdir(''.join([images_folder, c]))[1])
'''

blue 200
./dataset/images/blue DarkBlue_1_2_200 0 0 139 1.3
blue DarkBlue_1_2.png DarkBlue_1_2 .png 0 0 139
./dataset/images/blue Navy_1_0_low_201 0 0 128 1.3
blue Navy_1_0_low.png Navy_1_0_low .png 0 0 128
./dataset/images/blue Cyan_0_9_202 0 255 255 1.2
blue Cyan_0_9.png Cyan_0_9 .png 0 255 255
./dataset/images/blue MidnightBlue_1_3_low_203 25 25 112 1.1
blue MidnightBlue_1_3_low.png MidnightBlue_1_3_low .png 25 25 112
./dataset/images/blue DodgerBlue_1_1_204 30 144 255 1.3
blue DodgerBlue_1_1.png DodgerBlue_1_1 .png 30 144 255
./dataset/images/blue MidnightBlue_1_3_low_203_1_1_low_205 25 25 112 1.1
blue MidnightBlue_1_3_low_203_1_1_low.png MidnightBlue_1_3_low_203_1_1_low .png 25 25 112
./dataset/images/blue PaleTurquoise_1_2_low_206 175 238 238 1.4
blue PaleTurquoise_1_2_low.png PaleTurquoise_1_2_low .png 175 238 238
./dataset/images/blue PowderBlue_1_3_low_207 176 224 230 1.3
blue PowderBlue_1_3_low.png PowderBlue_1_3_low .png 176 224 230
./dataset/images/blue LightSkyBlue_1_1_low

./dataset/images/blue LightBlue_0_9_275 173 216 230 1.2
blue LightBlue_0_9.png LightBlue_0_9 .png 173 216 230
./dataset/images/blue DodgerBlue_1_1_204_1_3_low_220_1_2_low_237_1_4_low_238_1_0_low_276 30 144 255 0.9
blue DodgerBlue_1_1_204_1_3_low_220_1_2_low_237_1_4_low_238_1_0_low.png DodgerBlue_1_1_204_1_3_low_220_1_2_low_237_1_4_low_238_1_0_low .png 30 144 255
./dataset/images/blue LightBlue_1_3_226_1_3_low_277 173 216 230 1.0
blue LightBlue_1_3_226_1_3_low.png LightBlue_1_3_226_1_3_low .png 173 216 230
./dataset/images/blue DodgerBlue_1_2_low_217_1_1_278 30 144 255 1.1
blue DodgerBlue_1_2_low_217_1_1.png DodgerBlue_1_2_low_217_1_1 .png 30 144 255
./dataset/images/blue Blue_1_1_low_243_0_9_279 0 0 255 1.2
blue Blue_1_1_low_243_0_9.png Blue_1_1_low_243_0_9 .png 0 0 255
./dataset/images/blue Blue_1_1_low_243_0_9_low_280 0 0 255 1.2
blue Blue_1_1_low_243_0_9_low.png Blue_1_1_low_243_0_9_low .png 0 0 255
./dataset/images/blue Blue_1_0_281 0 0 255 1.0
blue Blue_1_0.png Blue_1_0 .png 0 0 2

blue Aqua_0_9_334_0_9.png Aqua_0_9_334_0_9 .png 0 255 255
./dataset/images/blue Blue_1_1_low_243_0_9_low_280_1_2_337 0 0 255 1.4
blue Blue_1_1_low_243_0_9_low_280_1_2.png Blue_1_1_low_243_0_9_low_280_1_2 .png 0 0 255
./dataset/images/blue Blue_1_1_low_243_0_9_307_1_3_338 0 0 255 1.3
blue Blue_1_1_low_243_0_9_307_1_3.png Blue_1_1_low_243_0_9_307_1_3 .png 0 0 255
./dataset/images/blue Cyan_1_3_299_1_0_339 0 255 255 1.3
blue Cyan_1_3_299_1_0.png Cyan_1_3_299_1_0 .png 0 255 255
./dataset/images/blue Aqua_1_0_low_260_1_1_low_340 0 255 255 1.3
blue Aqua_1_0_low_260_1_1_low.png Aqua_1_0_low_260_1_1_low .png 0 255 255
./dataset/images/blue DarkBlue_1_2_200_1_3_low_286_0_9_341 0 0 139 1.4
blue DarkBlue_1_2_200_1_3_low_286_0_9.png DarkBlue_1_2_200_1_3_low_286_0_9 .png 0 0 139
./dataset/images/blue Blue_1_1_low_342 0 0 255 1.0
blue Blue_1_1_low.png Blue_1_1_low .png 0 0 255
./dataset/images/blue Aqua_1_3_246_0_9_low_343 0 255 255 1.1
blue Aqua_1_3_246_0_9_low.png Aqua_1_3_246_0_9_low .png 0 255 2

blue Blue_1_1_low_243_0_9_307_1_3_low.png Blue_1_1_low_243_0_9_307_1_3_low .png 0 0 255
./dataset/images/blue Aqua_1_0_low_260_1_1_low_340_1_3_low_402_1_0_405 0 255 255 1.0
blue Aqua_1_0_low_260_1_1_low_340_1_3_low_402_1_0.png Aqua_1_0_low_260_1_1_low_340_1_3_low_402_1_0 .png 0 255 255
./dataset/images/blue Aqua_0_9_406 0 255 255 1.1
blue Aqua_0_9.png Aqua_0_9 .png 0 255 255
./dataset/images/blue Aqua_1_2_low_221_1_4_low_399_0_9_407 0 255 255 1.2
blue Aqua_1_2_low_221_1_4_low_399_0_9.png Aqua_1_2_low_221_1_4_low_399_0_9 .png 0 255 255
./dataset/images/blue Blue_1_1_low_243_0_9_279_1_2_low_354_1_0_360_1_3_408 0 0 255 1.0
blue Blue_1_1_low_243_0_9_279_1_2_low_354_1_0_360_1_3.png Blue_1_1_low_243_0_9_279_1_2_low_354_1_0_360_1_3 .png 0 0 255
./dataset/images/blue Aqua_1_0_low_288_0_9_335_1_2_409 0 255 255 1.4
blue Aqua_1_0_low_288_0_9_335_1_2.png Aqua_1_0_low_288_0_9_335_1_2 .png 0 255 255
./dataset/images/blue Aqua_1_2_235_1_3_low_304_1_4_low_309_1_1_low_410 0 255 255 1.2
blue Aqua_1_2_23

blue Aqua_1_1_290_1_3_low_432_1_3_low.png Aqua_1_1_290_1_3_low_432_1_3_low .png 0 255 255
./dataset/images/blue Aqua_1_1_290_1_3_423_0_9_low_472 0 255 255 1.0
blue Aqua_1_1_290_1_3_423_0_9_low.png Aqua_1_1_290_1_3_423_0_9_low .png 0 255 255
./dataset/images/blue Aqua_1_1_464_1_2_low_473 0 255 255 1.0
blue Aqua_1_1_464_1_2_low.png Aqua_1_1_464_1_2_low .png 0 255 255
./dataset/images/blue Aqua_1_2_235_1_3_low_304_1_4_low_309_1_1_low_322_1_2_low_378_1_0_low_441_1_3_474 0 255 255 1.1
blue Aqua_1_2_235_1_3_low_304_1_4_low_309_1_1_low_322_1_2_low_378_1_0_low_441_1_3.png Aqua_1_2_235_1_3_low_304_1_4_low_309_1_1_low_322_1_2_low_378_1_0_low_441_1_3 .png 0 255 255
./dataset/images/blue Aqua_1_0_low_260_1_1_329_1_4_440_1_2_475 0 255 255 1.0
blue Aqua_1_0_low_260_1_1_329_1_4_440_1_2.png Aqua_1_0_low_260_1_1_329_1_4_440_1_2 .png 0 255 255
./dataset/images/blue Aqua_0_9_334_0_9_336_1_3_low_458_1_1_low_476 0 255 255 0.9
blue Aqua_0_9_334_0_9_336_1_3_low_458_1_1_low.png Aqua_0_9_334_0_9_336_1_3_low_45

...

brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3 .png 165 42 42
./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_low_806_1_0_845_1_0_855 181 46 46 1.1
brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_low_806_1_0_845_1_0.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_low_806_1_0_845_1_0 .png 181 46 46
./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_856 165 42 42 1.3
brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2 .png 1

brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_low.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_low .png 181 46 46
./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_863_1_3_879 181 46 46 1.4
brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_863_1_3.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_79

./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_852_1_4_low_903 181 46 46 1.1
brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_852_1_4_low.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_852_1_4_low .png 181 46 46
./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_low_859_1_1_884_1_3_904 181 46 46 1.4
brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1

brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1 .png 165 42 42
./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_838_1_1_low_861_1_2_low_887_0_9_low_927 181 46 46 1.0
brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_838_1_1_low_861_1_2_low_887_0_9_low.png Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_838_1_1_low_861_1_2_low_887_0_9_low .png 181 46 46
./dataset/images/brown Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_

OSError: [Errno 22] Invalid argument: './dataset/images/brown/Brown_0_9_121_1_0_230_1_2_235_0_9_261_1_3_300_1_0_330_1_2_336_1_1_355_1_4_375_1_1_381_1_1_400_1_4_467_0_9_471_1_4_506_1_4_516_1_2_618_1_4_675_1_1_702_1_1_732_1_0_747_1_0_795_0_9_830_1_2_838_1_1_low_861_1_2_919_1_1_low_931_1_0_low_938_1_4_low_947_1_3_low.png'